# 處理實價登錄資料

In [1]:
import pandas as pd
import geopandas as gpd
import csv
import re
from shapely.geometry import Point

import sys, os
sys.path.append("..")
import my_function.full_to_half as htf
import my_function.write_coordinate_to_csv as wctc
print('Pandas version: ', pd.__version__)
print('Geopandas version: ', gpd.__version__)

Pandas version:  1.4.3
Geopandas version:  0.11.0


In [ ]:
path_text = r'../rawdata/real_price/'
path = os.listdir(path_text)
n = 0
for filename in path:
    try:
        if (filename[-3:] == 'csv') & (filename.split('_')[2] == 'a'):
            if n == 0:
                df1 = pd.read_csv(path_text + filename, encoding = 'utf-8')
                df1 = df1.iloc[1:]
                n = n + 1
            else:
                df2 = pd.read_csv(path_text + filename, encoding = 'utf-8')
                df2 = df2.iloc[1:]
                df1 = pd.concat([df1, df2])
    except:
        continue
#df1.info()
#df1.to_csv('df1.csv')

In [ ]:
# 處理電梯欄位
#print(df1)
filt = (df1['電梯'].isnull()) & (df1['建物型態'].str.contains('有電梯', na=False))
df1.loc[filt,['電梯']] = '有'
filt = (df1['電梯'].isnull())
df1.loc[filt,['電梯']] = '無'

In [ ]:
# 過濾特殊交易資料
filt = (df1['主要用途'] == '住家用') & (~df1['備註'].str.contains('親友', na=False)) & (~df1['備註'].str.contains('預售屋', na=False)) & (~df1['備註'].str.contains('地上權', na=False)) & (~df1['備註'].str.contains('夾層', na=False)) & ((df1['交易標的'] == '房地(土地+建物)') | (df1['交易標的'] == '房地(土地+建物)+車位'))
pre_df = df1.loc[filt,['土地位置建物門牌','鄉鎮市區','交易標的','交易年月日','建築完成年月','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','車位類別','電梯','單價元平方公尺']]
#pre_df = df1.loc[:,['土地位置建物門牌','交易年月日','建築完成年月','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','電梯','單價元平方公尺']]
pre_df = pre_df.dropna(subset=["單價元平方公尺"])
pre_df = pre_df.dropna(subset=["建築完成年月"])
pre_df.info()

In [ ]:
pre_df['屋齡']=pre_df['交易年月日'].str[0:3].astype(float)-pre_df['建築完成年月'].str[0:3].astype(float)

In [ ]:
pre_df['交易年份']=pre_df['交易年月日'].str[0:3]

In [ ]:
pre_df.reset_index(inplace=True, drop=False)
filt2 = (pre_df['屋齡'] >= 0)
pre_df = pre_df.loc[filt2,['交易年份','土地位置建物門牌','鄉鎮市區','屋齡','交易標的','交易年月日','建築完成年月','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','車位類別','電梯','單價元平方公尺']]
pre_df.info()

In [ ]:
pattern = r'[0-9]*?[零一二三四五六七八九十]*?[0-9]*?[零一二三四五六七八九十]{1}[0-9]*?[號]'
pre_df['修正地址'] = pre_df['土地位置建物門牌'].str.contains(pattern,regex=True)
pre_df['土地位置建物門牌'] = htf.h2f(pre_df['土地位置建物門牌'])
pre_df.to_csv('TPE_pre_df.csv')

In [ ]:
# 地址轉坐標
wctc.write('TPE_pre_df.csv', 'TPE_pre_df_WGS84.csv', 2,14)

In [2]:
df_final = pd.read_csv('./temp/TPE_pre_df_WGS84.csv', encoding = 'utf-8')
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42998 entries, 0 to 42997
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  42998 non-null  int64  
 1   交易年份        42998 non-null  int64  
 2   土地位置建物門牌    42998 non-null  object 
 3   鄉鎮市區        42998 non-null  object 
 4   屋齡          42998 non-null  float64
 5   交易標的        42998 non-null  object 
 6   交易年月日       42998 non-null  int64  
 7   建築完成年月      42998 non-null  int64  
 8   建物現況格局-房    42998 non-null  int64  
 9   建物現況格局-廳    42998 non-null  int64  
 10  建物現況格局-衛    42998 non-null  int64  
 11  車位類別        15775 non-null  object 
 12  電梯          42998 non-null  object 
 13  單價元平方公尺     42998 non-null  int64  
 14  修正地址        42998 non-null  bool   
 15  floor       42998 non-null  int64  
 16  lon         42931 non-null  float64
 17  lat         42931 non-null  float64
dtypes: bool(1), float64(3), int64(9), object(5)
memory usage: 5.6+ MB


In [3]:
df_final = df_final.dropna(subset=["lon"])
print(df_final.info())
geom = [Point(xy) for xy in zip(df_final.lon, df_final.lat)]
crs = {'init': 'epsg:4326'}
gf_final = gpd.GeoDataFrame(df_final, crs=crs, geometry=geom)
gf_final = gf_final.to_crs(epsg=3826)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42931 entries, 0 to 42997
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  42931 non-null  int64  
 1   交易年份        42931 non-null  int64  
 2   土地位置建物門牌    42931 non-null  object 
 3   鄉鎮市區        42931 non-null  object 
 4   屋齡          42931 non-null  float64
 5   交易標的        42931 non-null  object 
 6   交易年月日       42931 non-null  int64  
 7   建築完成年月      42931 non-null  int64  
 8   建物現況格局-房    42931 non-null  int64  
 9   建物現況格局-廳    42931 non-null  int64  
 10  建物現況格局-衛    42931 non-null  int64  
 11  車位類別        15766 non-null  object 
 12  電梯          42931 non-null  object 
 13  單價元平方公尺     42931 non-null  int64  
 14  修正地址        42931 non-null  bool   
 15  floor       42931 non-null  int64  
 16  lon         42931 non-null  float64
 17  lat         42931 non-null  float64
dtypes: bool(1), float64(3), int64(9), object(5)
memory usage: 5.9+ MB


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [4]:
gf_final = gf_final[['Unnamed: 0','鄉鎮市區', '交易年份', '屋齡', '交易標的', '建物現況格局-房', '建物現況格局-廳' ,'建物現況格局-衛', '車位類別', '電梯', 'floor', '單價元平方公尺', 'geometry']]
gf_final.loc[gf_final['交易標的'] == '房地(土地+建物)', '交易標的'] = 0
gf_final.loc[gf_final['交易標的'] == '房地(土地+建物)+車位', '交易標的'] = 1
gf_final.loc[gf_final['電梯'] == '無', '電梯'] = 0
gf_final.loc[gf_final['電梯'] == '有', '電梯'] = 1
# -----車位種類------
gf_final.loc[gf_final['車位類別'] == '坡道平面', '車位類別'] = 1
gf_final.loc[gf_final['車位類別'] == '坡道機械', '車位類別'] = 2
gf_final.loc[gf_final['車位類別'] == '升降平面', '車位類別'] = 3
gf_final.loc[gf_final['車位類別'] == '升降機械', '車位類別'] = 4
gf_final.loc[gf_final['車位類別'] == '一樓平面', '車位類別'] = 5
gf_final.loc[gf_final['車位類別'] == '塔式車位', '車位類別'] = 6
gf_final.loc[gf_final['車位類別'] == '其他' , '車位類別'] = 7
gf_final.loc[gf_final['車位類別'].isnull() , '車位類別'] = 0
print(gf_final.info())
print(gf_final.head())

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 42931 entries, 0 to 42997
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed: 0  42931 non-null  int64   
 1   鄉鎮市區        42931 non-null  object  
 2   交易年份        42931 non-null  int64   
 3   屋齡          42931 non-null  float64 
 4   交易標的        42931 non-null  object  
 5   建物現況格局-房    42931 non-null  int64   
 6   建物現況格局-廳    42931 non-null  int64   
 7   建物現況格局-衛    42931 non-null  int64   
 8   車位類別        42931 non-null  object  
 9   電梯          42931 non-null  object  
 10  floor       42931 non-null  int64   
 11  單價元平方公尺     42931 non-null  int64   
 12  geometry    42931 non-null  geometry
dtypes: float64(1), geometry(1), int64(7), object(4)
memory usage: 4.6+ MB
None
   Unnamed: 0 鄉鎮市區  交易年份   屋齡 交易標的  建物現況格局-房  建物現況格局-廳  建物現況格局-衛 車位類別 電梯  \
0           0  文山區   107  3.0    1         3         2         2    1  1   
1           1  萬華區   

In [5]:
gf_final.to_file('realprice' + ".geojson", driver='GeoJSON')

# 環域分析實價登錄資料-建置模型特徵資料

In [ ]:
df_final=pd.read_csv('pre_df_WGS84.csv',encoding='utf-8')
geom = [Point(xy) for xy in zip(df_final.lon, df_final.lat)]
crs = {'init': 'epsg:4326'}
gf_final = gpd.GeoDataFrame(df_final, crs=crs, geometry=geom)
gf_final = gf_final.to_crs(epsg=3826)
gf_final['ID'] = gf_final.index
gf_final = gf_final[['ID','交易年份','屋齡','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','電梯','單價元平方公尺','geometry']]
gf_final.info()
gf_final.plot()

In [ ]:
df_house=pd.read_csv('pre_df_WGS84.csv',encoding='utf-8')
geom = [Point(xy) for xy in zip(df_house.lon, df_house.lat)]
crs = {'init': 'epsg:4326'}
gf_house = gpd.GeoDataFrame(df_house, crs=crs, geometry=geom)
gf_house = gf_house.to_crs(epsg=3826)
gf_house['ID'] = gf_house.index
gf_house = gf_house[['ID','交易年份','屋齡','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','電梯','單價元平方公尺','geometry']]
gf_house.info()
gf_house.plot()

In [ ]:
gf_house_buf = gf_house
gf_house_buf['geometry'] = gf_house.buffer(500)
gf_house_buf.plot(alpha=0.1)

In [ ]:
df_hospital=pd.read_csv('./dataset/taiwan_hospital_getcoordinates.csv',encoding='utf-8')
geom = [Point(xy) for xy in zip(df_hospital.lon, df_hospital.lat)]
crs = {'init': 'epsg:4326'}
gf_hospital = gpd.GeoDataFrame(df_hospital, crs=crs, geometry=geom)
gf_hospital = gf_hospital.to_crs(epsg=3826)

In [ ]:
df_cstore=pd.read_csv('./dataset/taiwan_convenience_store_getcoordinates.csv',encoding='utf-8')
geom = [Point(xy) for xy in zip(df_cstore.lon, df_cstore.lat)]
crs = {'init': 'epsg:4326'}
gf_cstore = gpd.GeoDataFrame(df_cstore, crs=crs, geometry=geom)
gf_cstore = gf_cstore.to_crs(epsg=3826)

In [ ]:
# 讀取geojspn
gf_cemetery = gpd.read_file(f'./dataset/cemetery.geojson', encoding = 'utf-8')
gf_cemetery = gf_cemetery.to_crs(epsg=3826)

gf_fire_station = gpd.read_file(f'./dataset/fire_station.geojson', encoding = 'utf-8')
gf_fire_station = gf_fire_station.to_crs(epsg=3826)

gf_fuel = gpd.read_file(f'./dataset/fuel.geojson', encoding = 'utf-8')
gf_fuel = gf_fuel.to_crs(epsg=3826)

gf_library = gpd.read_file(f'./dataset/library.geojson', encoding = 'utf-8')
gf_library = gf_library.to_crs(epsg=3826)

gf_marketplace = gpd.read_file(f'./dataset/marketplace.geojson', encoding = 'utf-8')
gf_marketplace = gf_marketplace.to_crs(epsg=3826)

gf_park = gpd.read_file(f'./dataset/park.geojson', encoding = 'utf-8')
gf_park = gf_park.to_crs(epsg=3826)

gf_parking = gpd.read_file(f'./dataset/parking.geojson', encoding = 'utf-8')
gf_parking = gf_parking.to_crs(epsg=3826)

gf_place_of_worship = gpd.read_file(f'./dataset/place_of_worship.geojson', encoding = 'utf-8')
gf_place_of_worship = gf_place_of_worship.to_crs(epsg=3826)

gf_police = gpd.read_file(f'./dataset/police.geojson', encoding = 'utf-8')
gf_police = gf_police.to_crs(epsg=3826)

gf_school = gpd.read_file(f'./dataset/school.geojson', encoding = 'utf-8')
gf_school = gf_school.to_crs(epsg=3826)

gf_station = gpd.read_file(f'./dataset/station.geojson', encoding = 'utf-8')
gf_station = gf_station.to_crs(epsg=3826)

gf_university = gpd.read_file(f'./dataset/university.geojson', encoding = 'utf-8')
gf_university = gf_university.to_crs(epsg=3826)

In [ ]:
gf_university.info()

In [ ]:
gf_university['geometry'] = gf_university.buffer(0.001)
gf_university.plot()

In [ ]:
intersection = gpd.overlay(gf_house_buf,gf_university,  how='intersection', keep_geom_type=True)
#intersection['PARKING_AREA'] = intersection['geometry'].area
intersection_count = intersection.groupby('ID').agg({'id':'count'})
#intersection_count = intersection.groupby('ID').agg({'PARKING_AREA':'sum'})
intersection_count

In [ ]:
gf_final = gf_final.merge(intersection_count,how='left', left_on='ID', right_on='ID')

In [ ]:
gf_final.rename(columns={'id': 'NEAR_UNIVERSITY'}, inplace=True)
gf_final.loc[gf_final['NEAR_UNIVERSITY']>0, 'NEAR_UNIVERSITY'] = 1
gf_final['NEAR_UNIVERSITY'] = gf_final['NEAR_UNIVERSITY'].fillna(0)
print(gf_final.info())
gf_final

In [ ]:
gf_final.loc[gf_final['電梯']=='有', '電梯'] = 1
gf_final.loc[gf_final['電梯']=='無', '電梯'] = 0
gf_final

In [ ]:
gf_final = gf_final[['交易年份','屋齡','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','電梯','HOSPITAL_COUNT','CSTORE_COUNT','NEAR_UNIVERSITY','NEAR_FIRESTATION','NEAR_FUEL','NEAR_LIBRARY','NEAR_MARKET','PARK_AREA','PARKING_AREA','TEMPLE_COUNT','NEAR_POLICE','NEAR_SCHOOL','NEAR_STATION','單價元平方公尺']]
gf_final.to_csv('final_df.csv')